<a href="https://colab.research.google.com/github/adolfoguimaraes/datascience/blob/main/code/05_laboratorio_aprendizado_supervisionado_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Aprendizado Supervisionado

Vamos trabalhar com mais uma base e testar diferentes algortimos de Machine Learning. No entanto, nosso foco vai ser olhar as métricas de avaliação estudadas em sala de aula.

In [1]:
import pandas as pd
from tabulate import tabulate

## Base dos Dados 

Vamos utilizar a base de dados disponível nesse link: https://archive.ics.uci.edu/ml/datasets/spambase. A base está disponível na pasta `datasets` com o nome `SpamDataset.csv`. A versão deste repositório já está com os nomes das colunas. No repositório original, essas informações estão em arquivos separados. Não foi feita nenhuma modificação nos dados, apenas a inclusão no nome das colunas. 

In [2]:
data_ = pd.read_csv("../datasets/SpamDataset.csv")
data_

,word_freq_make,word_freq_address,word_freq_all,word_freq_3d,word_freq_our,word_freq_over,word_freq_remove,word_freq_internet,word_freq_order,word_freq_mail,...,char_freq_;,char_freq_(,char_freq_[,char_freq_!,char_freq_$,char_freq_#,capital_run_length_average,capital_run_length_longest,capital_run_length_total,classe
0,0.00,0.64,0.64,0.0,0.32,0.00,0.00,0.00,0.00,0.00,...,0.000,0.000,0.0,0.778,0.000,0.000,3.756,61,278,1
1,0.21,0.28,0.50,0.0,0.14,0.28,0.21,0.07,0.00,0.94,...,0.000,0.132,0.0,0.372,0.180,0.048,5.114,101,1028,1
2,0.06,0.00,0.71,0.0,1.23,0.19,0.19,0.12,0.64,0.25,...,0.010,0.143,0.0,0.276,0.184,0.010,9.821,485,2259,1
3,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.000,0.137,0.0,0.137,0.000,0.000,3.537,40,191,1
4,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.000,0.135,0.0,0.135,0.000,0.000,3.537,40,191,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4596,0.31,0.00,0.62,0.0,0.00,0.31,0.00,0.00,0.00,0.00,...,0.000,0.232,0.0,0.000,0.000,0.000,1.142,3,88,0
4597,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,...,0.000,0.000,0.0,0.353,0.000,0.000,1.555,4,14,0
4598,0.30,0.00,0.30,0.0,0.00,0.00,0.00,0.00,0.00,0.00,...,0.102,0.718,0.0,0.000,0.000,0.000,1.404,6,118,0
4599,0.96,0.00,0.00,0.0,0.32,0.00,0.00,0.00,0.00,0.00,...,0.000,0.057,0.0,0.000,0.000,0.000,1.147,5,78,0


A base que vamos trabalhar é usada para classificar textos como SPAM ou NÃO SPAM. Cada instância é um e-mail que foi classificado nestas duas classes. Cada instância é caracterizada por 57 atributos e 1 atributo de classe. Cada um dos 57 atributos tem relação com o conteúdo do texto usado para gerar a base. A classe pode assumir dois valores: 1 (texto classificado como Spam) e 0 (texto classificado como Não Spam).

Segue uma descrição dos atributos (em inglês) tirada da documentação da base:

```
The last column of 'spambase.data' denotes whether the e-mail was 
considered spam (1) or not (0), i.e. unsolicited commercial e-mail.  
Most of the attributes indicate whether a particular word or
character was frequently occuring in the e-mail.  The run-length
attributes (55-57) measure the length of sequences of consecutive 
capital letters.  For the statistical measures of each attribute, 
see the end of this file.  Here are the definitions of the attributes:

48 continuous real [0,100] attributes of type word_freq_WORD 
= percentage of words in the e-mail that match WORD,
i.e. 100 * (number of times the WORD appears in the e-mail) / 
total number of words in e-mail.  A "word" in this case is any 
string of alphanumeric characters bounded by non-alphanumeric 
characters or end-of-string.

6 continuous real [0,100] attributes of type char_freq_CHAR
= percentage of characters in the e-mail that match CHAR,
i.e. 100 * (number of CHAR occurences) / total characters in e-mail

1 continuous real [1,...] attribute of type capital_run_length_average
= average length of uninterrupted sequences of capital letters

1 continuous integer [1,...] attribute of type capital_run_length_longest
= length of longest uninterrupted sequence of capital letters

1 continuous integer [1,...] attribute of type capital_run_length_total
= sum of length of uninterrupted sequences of capital letters
= total number of capital letters in the e-mail

1 nominal {0,1} class attribute of type spam
= denotes whether the e-mail was considered spam (1) or not (0), 
i.e. unsolicited commercial e-mail.  
```

Para mais informações, acesse o link original da base. 

Vamos separar os atributos da classe nas variáveis `X` e `y`, respectivamente. 

In [3]:
X = data_[data_.columns[:-1]]
y = data_[data_.columns[-1]]

## Treinando os modelos 

Vamos treinar os modelos vistos em sala de aula: KNN, Árvore de Decisão e SVM. Em cada um destes modelos vamos alterar alguns atributos com o intuito de ajustar o melhor modelo. 

### KNN

Vamos usar a implementação [`sklearn.neighbors.KNeighborsClassifier`](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html) do KNN no Scikit-Learn. Neste caso, vamos modificar o parâmetro K (número de vizinhos), representado pelo atributo `n_neighbors`. A escolha do melhor valor de K depende dos dados que serão treinados. De forma geral, um valor alto de K reduz os efeitos de ruídos. No entanto, eles tornam os limites de classificação menos distintos, o que pode implicar em erros do modelo. 

### Árvore de Decisão 

Vamos usar a implementação [`sklearn.tree.DecisionTreeClassifier`](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html) da árvore de Decisão do Scikit-Learn. Neste exemplo, vamos trabalhar com dois parâmetros que nos ajudam a controlar o tamanho da árvore de decisão gerada. Lembre que em alguns casos, o tamanho da árvore pode ser um problema. Podar essa árvore pode evitar problemas de consumo de memória e, em alguns casos, evitar uma árvore superajustada que cause o que chamamos de `overfitting`. O tamanho da árvore vai ser controlado pelo parâmetro `max_depth` (A profundidade máxima da árvore. Se `None` (valor padrão), os nós são expandidos até todos os nós serem puros ou até todas as folhas conterem pelo menos `min_samples_split` exemplos). 

[Ver esse link](https://stackoverflow.com/questions/46480457/difference-between-min-samples-split-and-min-samples-leaf-in-sklearn-decisiontre) para entender um pouco melhor a diferença de `min_samples_split` e `min_samples_leaf`. 

### SVM 

Vamos usar a implementação [`sklearn.svm.SVC`](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html) do SVM do Scikit-Learn. Vamos modificar três parâmetros: 

* `kernel`: Especifica o tipo de kernel que queremos usar no algoritmo. O `rbf` é o padrão e pode assumir os valores `linear`, `poly`, `rbf`, `sigmoid`, `precomputed`. Vamos utilizar o `linear`, `poly` e o `rbf` nos nossos exemplos. 
* `C`: O `C` é um parâmetro de regularização que controla "o quanto o modelo erra". Valores altos implica que o modelo vai errar mais e valores baixos implica em menos erros. Não existe uma regra de que o menor erro implica em um modelo melhor. O parâmetro deve ser modificado e testado para a base de dados. 
* `gamma`: O `gamma` é um parâmetro utilizado quando trabalhamos com o kernel `rbf` que ajuda a controlar o quanto de curvatura vamos querer no modelo. Maior valor implica em maior curvaturas e menor, menos curvaturas. Isso ajuda a controlar o quão complexo é o modelo que está sendo gerado. 

[Veja esse link](https://medium.com/@myselfaman12345/c-and-gamma-in-svm-e6cee48626be) para entender um pouco mais destes parâmetros.


Vamos treinar nosso modelos :) 





## Usando Validação Cruzada

Vamos utilizar a validação cruzada para executar os testes dos modelos escolhidos. Para cada modelo, vamos mudar alguns parâmetros e analisar do ponto de vista das métricas a qualidade de cada modelo. 



In [4]:
def cross_validate_model(model, value_X, value_y):

    cross_result = cross_validate(model, value_X, value_y, scoring=('accuracy','precision','recall','f1'))

    result_values = [
        cross_result['fit_time'].mean(), 
        cross_result['score_time'].mean(), 
        cross_result['test_accuracy'].mean(), 
        cross_result['test_precision'].mean(), 
        cross_result['test_recall'].mean(), 
        cross_result['test_f1'].mean()
    ]

    return result_values

    

Vamos começar com o KNN, informando que o valor de K deve variar de 1 a 15. 

In [5]:
from sklearn.model_selection import cross_validate
from sklearn.neighbors import KNeighborsClassifier

In [6]:
all_knn_results = []
for k in range(1,15):

    # Validação Cruzada
    knn_ = KNeighborsClassifier(n_neighbors=k)
    results_ = cross_validate_model(knn_, X, y)

    # Gerar a lista de valores
    temp_list = [k]
    temp_list.extend(results_)
    all_knn_results.append(temp_list)


In [7]:
print(tabulate(all_knn_results, headers=['k','Tempo Treimo','Tempo Teste','Acurácia','Precisão','Recall','F1']))

  k    Tempo Treimo    Tempo Teste    Acurácia    Precisão    Recall        F1
---  --------------  -------------  ----------  ----------  --------  --------
  1      0.00836754       0.150531    0.777007    0.717829  0.724201  0.720072
  2      0.00512824       0.112086    0.764835    0.781242  0.563131  0.653822
  3      0.0049839        0.122019    0.776354    0.720975  0.719234  0.718433
  4      0.00458016       0.132687    0.767444    0.754983  0.613873  0.675613
  5      0.00441027       0.136835    0.772446    0.720053  0.699938  0.708679
  6      0.00559516       0.13261     0.760059    0.732517  0.62272   0.672297
  7      0.00395522       0.135565    0.768752    0.713367  0.70105   0.70594
  8      0.00434074       0.133412    0.759624    0.728924  0.628793  0.673932
  9      0.0047761        0.134825    0.759842    0.702385  0.687268  0.693561
 10      0.00540185       0.132385    0.759842    0.724332  0.639823  0.677938
 11      0.00471277       0.132832    0.760278    0.7

No caso da árvore de decisão vamos modificar o atributo que controla o tamanho da árvore. 

In [8]:
from sklearn.tree import DecisionTreeClassifier

In [9]:
all_tree_results = []
for depth_ in range(1, 15): 
    tree_ = DecisionTreeClassifier(max_depth=depth_, random_state=42)
    temp_list = [depth_]
    results_ = cross_validate_model(tree_, X, y)
    temp_list.extend(results_)
    all_tree_results.append(temp_list)

In [10]:
print(tabulate(all_tree_results, headers=['Profundidade','Tempo Treimo','Tempo Teste','Acurácia','Precisão','Recall','F1']))

  Profundidade    Tempo Treimo    Tempo Teste    Acurácia    Precisão    Recall        F1
--------------  --------------  -------------  ----------  ----------  --------  --------
             1       0.0392785      0.047644     0.77505     0.825201  0.591349  0.677385
             2       0.0564623      0.0256186    0.840902    0.873568  0.73528   0.790326
             3       0.116419       0.0408113    0.869814    0.873883  0.791515  0.827812
             4       0.0962426      0.0449592    0.881762    0.900529  0.796452  0.843372
             5       0.140757       0.0229484    0.889586    0.886307  0.836175  0.858561
             6       0.103733       0.0247947    0.899148    0.893452  0.853804  0.871505
             7       0.0843343      0.015084     0.899799    0.886041  0.867606  0.874813
             8       0.0942235      0.0149621    0.896538    0.882163  0.864849  0.871281
             9       0.100732       0.0152923    0.895018    0.880424  0.863197  0.869354
          

No caso do SVM vamos modificar os atributos `C` e `gamma` que controla as funções de separação que são geradas. 

In [ ]:
from sklearn.svm import SVC 

O `C` vai variar com os valores `[0.01, 0.1, 1, 10, 100, 1000]`.

In [ ]:
all_svm_results = []
for C in [0.01, 0.1, 1, 10, 100, 1000]:
    print("C = %f" % C, end=" ")
    svm = SVC(kernel='rbf', C=C, random_state=42)
    temp_list = [C]
    result_ = cross_validate_model(svm, X, y)
    temp_list.extend(result_)
    all_svm_results.append(temp_list)



C = 0.010000 C = 0.100000 C = 1.000000 C = 10.000000 C = 100.000000 C = 1000.000000 

In [ ]:
print(tabulate(all_svm_results, headers=['C','Tempo Treimo','Tempo Teste','Acurácia','Precisão','Recall','F1']))

      C    Tempo Treimo    Tempo Teste    Acurácia    Precisão    Recall        F1
-------  --------------  -------------  ----------  ----------  --------  --------
   0.01        1.06864        0.781397    0.668129    0.686506  0.301167  0.417589
   0.1         0.866678       0.678889    0.69008     0.670696  0.424713  0.51959
   1           0.948487       0.6499      0.705075    0.706021  0.43849   0.539785
  10           0.86999        0.638836    0.732235    0.745362  0.493119  0.592809
 100           0.828926       0.584946    0.821997    0.81729   0.724242  0.764404
1000           1.15679        0.437789    0.885458    0.871563  0.847211  0.856721


O `gamma` vai variar com os valores `[0.001, 0.01, 0.1, 1, 10, 100, 1000]`

In [ ]:
all_svm_results = []
for gamma in [0.001, 0.01, 0.1, 1, 10, 100, 1000]:
    print("Gamma = %.3f" % gamma, end=" ")
    svm = SVC(kernel='rbf', C=1000, gamma=gamma, random_state=42)
    temp_list = [gamma]
    result_ = cross_validate_model(svm, X, y)
    temp_list.extend(result_)
    all_svm_results.append(temp_list)



Gamma = 0.001 Gamma = 0.010 Gamma = 0.100 Gamma = 1.000 Gamma = 10.000 Gamma = 100.000 Gamma = 1000.000 

In [ ]:
print(tabulate(all_svm_results, headers=['C','Tempo Treimo','Tempo Teste','Acurácia','Precisão','Recall','F1']))

       C    Tempo Treimo    Tempo Teste    Acurácia    Precisão    Recall        F1
--------  --------------  -------------  ----------  ----------  --------  --------
   0.001         2.25774       0.374322    0.859811    0.822488  0.838927  0.828333
   0.01          1.31231       0.645213    0.824821    0.761331  0.820176  0.788416
   0.1           1.38736       0.967566    0.726141    0.779357  0.424081  0.548392
   1             1.16926       1.00877     0.677682    0.989387  0.184196  0.307747
  10             1.21665       1.05491     0.664858    0.994286  0.150564  0.259727
 100             1.0731        0.95421     0.661164    0.994118  0.141187  0.24548
1000             1.07908       0.996343    0.655079    0.99375   0.125743  0.220919


## Agora é como vocês

Analise os resultados gerados até aqui juntamente com mais dois métodos: Regressão Logística e RandomForest. Pesquise sobre esses métodos e veja quais atributos seriam interessantes de modificar. Execute os testes e escolha dentre todos os modelos analisados qual é o melhor para a tarefa de detecção de SPAM e NÃO SPAM. 

* Documentação da Regressão Logística: https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html
* Documentação do Random Forest: https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html